# SariCoach: Agentic Retail Coach for Sari-Sari Stores
**Agents Intensive Capstone Project**

This notebook demonstrates the **SariCoach** system, a multi-agent AI designed to provide actionable coaching to micro-retailers.

### Course Concepts Demonstrated:
1.  **Multi-agent System:** `PlannerAgent` orchestrates `DataAnalystAgent` and `CoachAgent`.
2.  **Tools:** Custom tools for data loading, feature engineering, and correlation analysis.
3.  **Evaluation:** Structured evaluation harness with keyword coverage metrics.
4.  **Observability:** Structured logging of agent actions and outputs.

---

In [ ]:
# Install dependencies (if running in a fresh environment)
!pip install -q -r requirements.txt

In [ ]:
import json
import pandas as pd
from pathlib import Path
from src.feature_frame import DataContext
from src.agents.core import PlannerAgent, DataAnalystAgent, CoachAgent

BASE = Path("data/processed")

# Load canonical SariCoach dataset
try:
    brands = pd.read_csv(BASE / "brands.csv")
    products = pd.read_csv(BASE / "products.csv")
    stores = pd.read_csv(BASE / "stores.csv")
    transactions = pd.read_csv(BASE / "transactions.csv", parse_dates=["tx_timestamp"])
    transaction_lines = pd.read_csv(BASE / "transaction_lines.csv")
    shelf_vision = pd.read_csv(BASE / "shelf_vision_events.csv", parse_dates=["event_timestamp"])
    stt_events = pd.read_csv(BASE / "stt_events.csv", parse_dates=["event_timestamp"])
    weather = pd.read_csv(BASE / "weather_daily.csv", parse_dates=["date"])
    foot_traffic = pd.read_csv(BASE / "foot_traffic_daily.csv", parse_dates=["date"])
    
    print("✅ Loaded processed data from data/processed/")
    
    # Build the shared context
    ctx = DataContext(
        brands=brands,
        products=products,
        stores=stores,
        transactions=transactions,
        transaction_lines=transaction_lines,
        shelf_vision=shelf_vision,
        stt_events=stt_events,
        weather=weather,
        foot_traffic=foot_traffic,
    )
    
    # Initialize Agents
    planner = PlannerAgent()
    analyst = DataAnalystAgent(ctx=ctx)
    coach = CoachAgent(model_name="heuristic", use_gemini=False)
    
    print("✅ SariCoach Agents Initialized (Planner, Analyst, Coach)")
    
except FileNotFoundError:
    print("⚠️ Processed data not found. Please run `python3 seed_saricoach_data.py` first.")
    # Fallback for demo if data missing
    ctx = None

### Data Pipeline Overview

Before we build and run the agents, we start from a **canonical SariCoach dataset**
generated by `seed_saricoach_data.py`.

The pipeline:

- Ingests Kaggle-style retail data (store sales + orders + order items + products + stores).
- Normalizes it into:
  - `brands`, `products`, `stores`
  - `transactions`, `transaction_lines`
- Synthesizes multimodal signals on top:
  - `shelf_vision_events.csv` – facings, share of shelf, out-of-stock flags per store/brand/day
  - `stt_events.csv` – synthetic Taglish Speech-to-Text brand mentions with intent + sentiment
  - `weather_daily.csv` – temperature, rainfall, weather condition
  - `foot_traffic_daily.csv` – daily traffic index per store

In this notebook, we **only read from `data/processed/*.csv`** so the entire demo is
self-contained and reproducible. The agents treat these tables as their world model:
the DataAnalystAgent builds feature frames from them, and the CoachAgent turns those
frames into concrete 7-day action plans for sari-sari store owners.

## 2. Run Agent Loop
We simulate a user request for a "7-day action plan" for a specific store. The Planner routes this to the Analyst, who builds the feature frame, and then the Coach generates advice.

In [ ]:
if ctx:
    # Pick a random store to analyze
    sample_store_id = int(stores.sample(1, random_state=42)["store_id"].iloc[0])
    store_name = stores[stores["store_id"] == sample_store_id]["store_name"].iloc[0]
    
    print(f"🤖 Analyzing: {store_name} (ID: {sample_store_id})")
    
    # 1. Planner decides what to do
    decision = planner.plan({
        "type": "seven_day_plan",
        "store_id": sample_store_id,
        "days": 30,
        "category": None,  # or "Beverages" etc.
    })
    
    # 2. Analyst builds features and metrics
    analytics = analyst.analyze(decision)
    
    # 3. Coach generates advice
    coach_output = coach.coach(analytics, persona="store_owner")
    
    print("\n=== Recommended Actions (next 7 days) ===")
    for a in coach_output.actions:
        print(f"- {a}")

    print("\n=== Risks ===")
    for r in coach_output.risks:
        print(f"- {r}")

    print("\n=== Opportunities ===")
    for o in coach_output.opportunities:
        print(f"- {o}")

    print("\n[Debug]", coach_output.debug_notes)
else:
    print("Skipping agent loop due to missing data.")

## 3. Evaluation
We run the evaluation harness against a set of test scenarios.

In [ ]:
!make eval